In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [2]:
# also we need NMS (to use only selected bounding box)
# kalman filters
# tracker
# model for detection

In [3]:
from absl import flags
import sys
tf.compat.v1.flags.DEFINE_string('f','','')
FLAGS = flags.FLAGS
FLAGS(sys.argv)

['C:\\Users\\srihari_mohan\\Anaconda3\\envs\\deepsort\\lib\\site-packages\\ipykernel_launcher.py']

In [4]:
from deep_sort.detection import Detection
from deep_sort import nn_matching
from deep_sort.tracker import Tracker
from deep_sort.preprocessing import non_max_suppression
from tools import generate_detections as gdet

In [5]:
from yolov3_tf2.dataset import transform_images
from yolov3_tf2.utils import load_darknet_weights, convert_boxes
from yolov3_tf2.models import YoloV3

In [6]:
with open('data/labels/coco.names','rb') as f:
    class_names = f.read().decode()
    class_names = class_names.split("\r\n")
    class_names = class_names[:-1]

In [7]:
# Load YOLO using the weights
# load the model file pb mars-small128.pb (feature extractor from bounding box)
# Need to evaluate features so needed some distance metric for similarity
yolo = YoloV3(classes=len(class_names))
yolo.load_weights("weights/yolov3.tf")

In [8]:
feature_model = r"model_data/mars-small128.pb"
encoder = gdet.create_box_encoder(feature_model)
# Define a metric (cosine)
cosine_threshold = 0.5
nn_budget = None
metric = nn_matching.NearestNeighborDistanceMetric("cosine", cosine_threshold, budget=nn_budget)
tracker = Tracker(metric)

In [9]:
# So now yolo detection done
# encoder extracts features
# tracker compares the features between t

In [10]:
vid = cv2.VideoCapture('data/video/cars.mp4')

codec = cv2.VideoWriter_fourcc(*'XVID')
vid_fps =int(vid.get(cv2.CAP_PROP_FPS))
vid_width,vid_height = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./data/video/results.avi', codec, vid_fps, (vid_width, vid_height))

from _collections import deque
pts = [deque(maxlen=30) for _ in range(1000)]

counter = []

In [11]:
_, img = vid.read()
print(type(img))

<class 'numpy.ndarray'>


In [12]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [13]:
img = np.expand_dims(img, axis=0)

In [14]:
img.shape

(1, 1080, 1920, 3)

In [15]:
img = transform_images(img, 416)

In [16]:
img.shape

TensorShape([1, 416, 416, 3])

In [17]:
box, score, classes, nums = yolo.predict(img)

In [18]:
classes = classes[0]

In [19]:
classes

array([2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 2., 7., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [20]:
names = []
for i in range(len(classes)):
    names.append(class_names[int(classes[i])])
names = np.array(names)

In [21]:
names

array(['car', 'car', 'car', 'car', 'car', 'car', 'car', 'car', 'person',
       'person', 'car', 'truck', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'person', 'person',
       'person', 'person', 'person', 'person', 'pe

In [22]:
# pass in each box and extract features

In [23]:
converted_box = convert_boxes(img, box[0])

In [24]:
features = encoder(img, converted_box)

In [25]:
detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in
                zip(converted_box, score[0], names, features)]

In [26]:
boxs = np.array([d.tlwh for d in detections])
scores = np.array([d.confidence for d in detections])
classes = np.array([d.class_name for d in detections])
nms_max_overlap = 0.8
indices = non_max_suppression(boxs, classes, nms_max_overlap, scores)
detections = [detections[i] for i in indices]

In [27]:
tracker.predict()
tracker.update(detections)

C:\Users\srihari_mohan\Documents\PersonalProjects\DeepSortTracking\Single-Multiple-Custom-Object-Detection-and-Tracking-master\deep_sort\detection.py:54: RuntimeWarning: divide by zero encountered in double_scalars
  ret[2] /= ret[3]


In [28]:
tracker

In [32]:
for track in tracker.tracks:
    bbox = track.to_tlbr()
    class_name= track.get_class()
    print(bbox)

[nan  0. nan  0.]
[nan  0. nan  0.]
[nan  0. nan  0.]
[nan  0. nan  0.]
[nan  0. nan  0.]
[nan  0. nan  0.]
[nan  0. nan  0.]
[nan  0. nan  0.]


In [31]:
class_name

'car'